# Images
We used lupton's algorithm to compute RGB images from our data. The data was aligned taking green as the true position, this is chosen arbitrarily.

In [ ]:
from ObservationData import ObservationData
import numpy as np
from astropy.visualization import make_lupton_rgb
import os
import matplotlib.pyplot as plt
from astroalign import register
from astropy.io import fits
from skimage import exposure, color, img_as_float, restoration

data_dir = "/net/virgo01/data/users/tboersma/2025-04-30/final_images"

print("Loading data...")
red = fits.getdata(os.path.join(data_dir, "final_M81_R*.fits"))
green = fits.getdata(os.path.join(data_dir, "final_M81_G*.fits"))
blue = fits.getdata(os.path.join(data_dir, "final_M81_B*.fits"))



def normalize(data):
    return data / np.percentile(data, 99.5)

# green as true
print("Aligning Images...")
ref = green.byteswap().newbyteorder() 
red, _ = register(red.byteswap().newbyteorder() , ref)
blue, _ = register(blue.byteswap().newbyteorder() , ref)


# Normalize channels
print("Normalizing Channels...")
r_norm = normalize(red)
g_norm = normalize(green)
b_norm = normalize(blue)

# Create RGB image using Lupton's algorithm (asinh stretch, good for astronomy)

rgb_image = make_lupton_rgb(r_norm, g_norm, b_norm, Q=10, stretch=0.5)

# Plot the result
plt.figure(figsize=(10, 10))
plt.imshow(rgb_image, origin='lower')
plt.title("RGB Composite", fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()